In [40]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np

In [41]:
df_F1= pd.read_csv("../data/F1_4_Detailed releases at facility level with E-PRTR Sector and Annex I Activity detail into Air.csv")
print(df_F1.shape)
print(df_F1.describe())
df_F1.head(5)

(288566, 14)
       EPRTRSectorCode      Longitude       Latitude     emissions  \
count    288566.000000  288566.000000  288566.000000  2.885660e+05   
mean          4.205239       7.248907      48.293419  9.502748e+07   
std           2.513370      10.538875       6.933106  6.984015e+08   
min           1.000000     -63.080350     -21.303590  0.000000e+00   
25%           1.000000      -0.915578      43.575665  1.160000e+04   
50%           4.000000       6.709500      49.210195  4.120000e+04   
75%           7.000000      14.574320      52.211600  4.950000e+05   
max           9.000000      55.666840      76.597755  4.320000e+10   

       reportingYear  
count  288566.000000  
mean     2013.016676  
std         3.891024  
min      2007.000000  
25%      2010.000000  
50%      2013.000000  
75%      2016.000000  
max      2020.000000  


,countryName,EPRTRSectorCode,eprtrSectorName,EPRTRAnnexIMainActivityCode,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,Longitude,Latitude,City,targetRelease,pollutant,emissions,reportingYear
0,Austria,4,Chemical industry,4(d),Chemical installations for the production on a...,AT.CAED/9008390392737.FACILITY,Nufarm GmbH & CO KG,14.336056,48.290943,Linz,AIR,"1,2-dichloroethane (DCE-1,2)",3300.0,2007
1,Austria,9,Other activities,9(c),Installations for the surface treatment of sub...,AT.CAED/9008390396414.FACILITY,MAGNA STEYR Fahrzeugtechnik AG&Co KG,15.477778,47.019170,"Graz,07.Bez.:Liebenau",AIR,Non-methane volatile organic compounds (NMVOC),170000.0,2007
2,Austria,6,Paper and wood production and processing,6(b),Industrial plants for the production of paper ...,AT.CAED/9008390397176.FACILITY,Fritz EGGER GmbH & Co. OG,15.691833,48.259556,Unterradlberg,AIR,Carbon dioxide (CO2),228000000.0,2007
3,Austria,6,Paper and wood production and processing,6(b),Industrial plants for the production of paper ...,AT.CAED/9008390397176.FACILITY,Fritz EGGER GmbH & Co. OG,15.691833,48.259556,Unterradlberg,AIR,Nitrogen oxides (NOX),311000.0,2007
4,Austria,3,Mineral industry,3(c),Installations for the production of cement cli...,AT.CAED/9008390397220.FACILITY,voestalpine Stahl GmbH,14.159778,47.807083,Klaus an der Pyhrnbahn,AIR,Carbon dioxide (CO2),329000000.0,2007


Shape = (288566 rows, 14 columns)
* There is null values for the emission
* targetRelease column irrelevant (same value for all columns)


### Drop targetRelease column

In [42]:
df_F1 = df_F1.drop(columns='targetRelease',axis=1)

95 null values of emission : drop or imputation or search in another data ?
from which value we keep the emission ?

### Replace "0" by "NaN"

In [43]:
df_F1.emissions = df_F1.emissions.replace(0,np.nan)

In [20]:
df_CLR = pd.read_csv("../data/clean_clrtap.csv",sep='\t')
df_CLR.head()

,Country_Code,Country,Pollutant_name,Format_name,Sector_code,Year,Emissions,Notation,VersionId,Parent_sector_code,Sector_name,Sector_label_EEA
0,AT,Austria,As,NFR 2014-1,1B2aiv,1990,NaN,NR,2,NATIONAL TOTAL,Fugitive emissions oil: Refining / storage,Energy production and distribution
1,AT,Austria,As,NFR 2014-1,1B2av,1990,NaN,NR,2,NATIONAL TOTAL,Distribution of oil products,Energy production and distribution
2,AT,Austria,As,NFR 2014-1,1B2b,1990,NaN,NR,2,NATIONAL TOTAL,Fugitive emissions from natural gas (explorati...,Energy production and distribution
3,AT,Austria,As,NFR 2014-1,1B2c,1990,NaN,NR,2,NATIONAL TOTAL,"Venting and flaring (oil, gas, combined oil an...",Energy production and distribution
4,AT,Austria,As,NFR 2014-1,1B2d,1990,NaN,NR,2,NATIONAL TOTAL,Other fugitive emissions from energy production,Energy production and distribution


# Merging Data

#### Make columns matchable

Pollutant

In [44]:
pollutant_CLR = list(df_CLR.Pollutant_name.unique())
pollutant_F1 = list(df_F1.pollutant.unique())

print(pollutant_CLR)
print(pollutant_F1,end='\n')

def cleanF1Pollutant():
    pollutant_F1_clean = pollutant_F1.copy()
    for k,v in enumerate(pollutant_F1_clean):
        searchObj = re.search('(?<=\()[-,a-zA-Z0-9 ]*(?=\))', v)
        if searchObj:
             temp = searchObj.group().replace('as ', '')
             pollutant_F1_clean[k] = temp
    pollutant_F1_clean[pollutant_F1.index('Benzo(g,h,i)perylene')] = 'Benzo(g,h,i)perylene'
    pollutant_F1_clean[pollutant_F1.index('PCDD + PCDF (dioxins + furans) (as Teq)')] = 'Teq'

    for i in pollutant_CLR:
        for k,v in enumerate(pollutant_F1_clean):
            if v.lower() in i.lower()  :
                pollutant_F1_clean[k]=i;

    return pollutant_F1_clean

df_F1.pollutant = df_F1.pollutant.replace(pollutant_F1,cleanF1Pollutant())
print(df_F1.pollutant.unique())

['As', 'BC', 'NH3', 'benzo(b)', 'benzo(k)', 'Cd', 'CO', 'Cu', 'Cr', 'dioxin', 'HCB', 'Hg', 'Ni', 'NOx', 'NMVOC', 'PCB', 'Pb', 'Se', 'SOx', 'PM10', 'total PAH', 'Zn', 'TSP', 'PM2.5', 'Indeno', 'benzo(a)']
['1,2-dichloroethane (DCE-1,2)', 'Non-methane volatile organic compounds (NMVOC)', 'Carbon dioxide (CO2)', 'Nitrogen oxides (NOX)', 'Sulphur oxides (SOX)', 'Zinc and compounds (as Zn)', 'Carbon monoxide (CO)', 'Methane (CH4)', 'Mercury and compounds (as Hg)', 'Hydrogen cyanide (HCN)', 'Particulate matter (PM10)', 'Nitrous oxide (N2O)', 'Hydrochlorofluorocarbons (HCFCs)', 'Chromium and compounds (as Cr)', 'Copper and compounds (as Cu)', 'Lead and compounds (as Pb)', 'Chlorine and inorganic compounds (as HCl)', 'Ammonia (NH3)', 'Benzene', 'Polycyclic aromatic hydrocarbons (PAHs)', 'Dichloromethane (DCM)', 'Cadmium and compounds (as Cd)', 'Nickel and compounds (as Ni)', 'PCDD + PCDF (dioxins + furans) (as Teq)', 'Hydro-fluorocarbons (HFCS)', 'Fluorine and inorganic compounds (as HF)', 'Na

Sector Name

In [14]:
print(df_CLR.Sector_label_EEA.unique())
print(df_F1.eprtrSectorName.unique())
df_CLR['Sector_label_EEA'] = df_CLR['Sector_label_EEA'].replace(['Energy production and distribution','Energy use in industry'],'Energy sector')
df_CLR['Sector_label_EEA'] = df_CLR['Sector_label_EEA'].replace(['Other'],['Other activities'])
df_F1['eprtrSectorName'] = df_F1['eprtrSectorName'].replace(['Waste and wastewater management','Animal and vegetable products from the food and beverage sector'],['Waste','Agriculture'])

['Energy production and distribution'
 'Industrial processes and product use' 'Waste'
 'National total for the entire territory' 'Other'
 'Energy use in industry' 'Non-road transport' 'Road transport'
 'Commercial, institutional and households' 'Agriculture']
['Chemical industry' 'Other activities'
 'Paper and wood production and processing' 'Mineral industry'
 'Animal and vegetable products from the food and beverage sector'
 'Waste and wastewater management' 'Production and processing of metals'
 'Energy sector' 'Intensive livestock production and aquaculture']


Rename columns

In [15]:
print(df_CLR.columns)
print(df_F1.columns)
df_F1 = df_F1.rename(columns={'countryName':'Country',
                    'EPRTRSectorCode':'Sector_Code',
                    'eprtrSectorName':'Sector_Name',
                    'EPRTRAnnexIMainActivityLabel':'Sector_Label',
                    'emissions':'Emissions',
                    'reportingYear':'Year',
                    'pollutant':'Pollutant_Name',
                    'EPRTRAnnexIMainActivityCode':'Main_Activity_Code'
                    })
df_CLR = df_CLR.rename(columns={'Sector_label_EEA':'Sector_Name',
                    'Sector_name':'Sector_Label',
                     'Pollutant_name':'Pollutant_Name'
                    })

Index(['Country_Code', 'Country', 'Pollutant_name', 'Format_name',
       'Sector_code', 'Year', 'Emissions', 'Notation', 'VersionId',
       'Parent_sector_code', 'Sector_name', 'Sector_label_EEA'],
      dtype='object')
Index(['countryName', 'EPRTRSectorCode', 'eprtrSectorName',
       'EPRTRAnnexIMainActivityCode', 'EPRTRAnnexIMainActivityLabel',
       'FacilityInspireID', 'facilityName', 'Longitude', 'Latitude', 'City',
       'pollutant', 'emissions', 'reportingYear'],
      dtype='object')


Cities
* the column "city" is an additional column from the data F1. Indeed, we need the country, the sector and the city to get the emission, in the other we only need the country and the sector.
* Solution : ?

In [ ]:
df_F1.City.unique()

In [16]:
df_F1.City.unique()

array(['Linz', 'Graz,07.Bez.:Liebenau', 'Unterradlberg', ...,
       'Millerhill, Dalkeith', 'Port William, Newton Stewart',
       'Locherbie'], dtype=object)